In [17]:
path = r"Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv"
scene = 'us-101'

from highway_env.data.ngsim import *

reader = ngsim_data(scene)

In [27]:
print('Processing raw data...')
vr_dict = {}
snap_dict = {}
veh_dict = {}

# If your code elsewhere relies on these:
# NUM_COLS should be 25 and GLB_loc_colidx 24 for files with "Location"
# but we’ll infer from header so this works for both 24/25 cols.
with open(path, 'r', newline='', encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    header = next(reader, None)
    
    print(header)
    if not header:
        raise ValueError
    

    # Normalize header names and map to indices
    header = [h.strip().strip('"') for h in header]
    colidx = {name: i for i, name in enumerate(header)}
    # Detect optional Location column
    loc_idx = colidx.get('Location')
    # If your code uses self.name for scene filtering (e.g., 'peachtree'), keep it
    want_loc = ("us-101" or "").strip().lower()

    expected_cols = len(header)

    counter = 0 
    for row in reader:
        if not row:
            continue
        
        # Fix common trailing-comma -> empty-field case
        if len(row) == expected_cols + 1 and row[-1] == '':
            row = row[:-1]
        
        if len(row) != expected_cols:
            print (f"Row {counter+1}: expected {expected_cols} cols, got {len(row)}")
            raise ValueError

        # Filter by Location if present
        if loc_idx is not None and want_loc:
            loc_val = row[loc_idx].strip().strip('"').lower()

            if loc_val != want_loc:
                counter += 1
                continue

        norm = row[:]
        
        for i in range(expected_cols):
            
            if i == loc_idx:
                continue
                
            norm[i] = norm[i].strip().strip('"').replace(',', '')

        # Recreate a CSV-like line if build_from_raw expects a single string
        line_str = ','.join(norm)
        print(line_str)
        

        tmp_vr = vehicle_record()
        
        tmp_vr.build_from_raw(counter, line_str)
        
        counter += 1 
        vr_dict[tmp_vr.ID] = tmp_vr
        
        print(vr_dict[tmp_vr.ID])

        if tmp_vr.unixtime not in snap_dict:
            snap_dict[tmp_vr.unixtime] = snapshot(tmp_vr.unixtime)
        
        snap_dict[tmp_vr.unixtime].add_vr(tmp_vr)

        print(snap_dict[tmp_vr.unixtime])

        if tmp_vr.veh_ID not in veh_dict:
            veh_dict[tmp_vr.veh_ID] = vehicle(tmp_vr.veh_ID)
        
        veh_dict[tmp_vr.veh_ID].add_vr(tmp_vr)
        print(veh_dict)

    
        

        break

        


Processing raw data...
['Vehicle_ID', 'Frame_ID', 'Total_Frames', 'Global_Time', 'Local_X', 'Local_Y', 'Global_X', 'Global_Y', 'v_length', 'v_Width', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'O_Zone', 'D_Zone', 'Int_ID', 'Section_ID', 'Direction', 'Movement', 'Preceding', 'Following', 'Space_Headway', 'Time_Headway', 'Location']
515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13,6.9,2,23.31,2.05,3,NA,NA,NA,NA,NA,NA,500,523,119.1,5.11,us-101
Vehicle record: 0, vehicle ID: 515, unixtime: 1118848075000, time: 2005-06-15 08:07:55, lane: 3, y: 188.062, x: 30.034
Snapshot: unixtime: 1118848075000, number of vehs: 1
{np.int64(515): Vehicle: veh_ID: 515, number of unixtimes: 1}


ValueError: I/O operation on closed file.